In [6]:
# Force reload of updated filtering modules
import importlib
import sys

# Reload all filtering modules to get latest changes
if 'sledhead_imu' in sys.modules:
    importlib.reload(sys.modules['sledhead_imu'])
    importlib.reload(sys.modules['sledhead_imu.filtering'])
    importlib.reload(sys.modules['sledhead_imu.filtering.high_pass'])
    importlib.reload(sys.modules['sledhead_imu.filtering.low_pass'])
    importlib.reload(sys.modules['sledhead_imu.filtering.median'])
    importlib.reload(sys.modules['sledhead_imu.filtering.rolling_avg'])
    importlib.reload(sys.modules['sledhead_imu.filtering.comprehensive'])

print("✅ Filtering modules reloaded with latest changes!")


✅ Filtering modules reloaded with latest changes!


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
sys.path.append('../../src')

from sledhead_imu.filtering.comprehensive import apply_filtering_pipeline

# Load daily aggregated data
data_dir = Path('../data')
daily_dir = data_dir / '04_aggregate_per_day' / 'daily_athlete_data'
filtered_dir = data_dir / '05_filtering' / 'filtered_data'

# Find daily aggregated files
daily_files = list(daily_dir.glob('*.csv'))
print(f"Found {len(daily_files)} daily aggregated files")

if daily_files:
    # Load first file as example
    df = pd.read_csv(daily_files[0])
    print(f"Daily data shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Apply comprehensive filtering pipeline
    print("\nApplying comprehensive filtering pipeline...")
    print("Parameters optimized for head-inclusive motion at 2000 Hz:")
    print("- High-pass Butterworth: 0.3 Hz, order = 2")
    print("- Low-pass Butterworth: 30 Hz, order = 4")
    print("- Median filter: kernel = 11 samples (~5.5 ms)")
    print("- Savitzky-Golay: window = 51 samples (~25 ms), polyorder = 3")
    print("- All filters use filtfilt for zero-phase filtering")
    
    df_filtered = apply_filtering_pipeline(
        df, 
        sampling_rate=2000.0,
        columns=['x_mean', 'y_mean', 'z_mean']
    )
    
    print(f"Final filtered shape: {df_filtered.shape}")
    
    # Save filtered data
    filtered_dir.mkdir(parents=True, exist_ok=True)
    output_file = filtered_dir / f"filtered_{daily_files[0].stem}.csv"
    df_filtered.to_csv(output_file, index=False)
    print(f"Saved filtered data to: {output_file}")
    
    # Show sample of filtered data
    print("\nSample filtered data:")
    print(df_filtered.head())
    
else:
    print("No daily aggregated data found. Using sample data for demonstration...")
    
    # Load sample data directly for demo
    sample_files = list((data_dir / '00_collect' / 'imu').glob('sample_imu_*.csv'))
    if sample_files:
        df = pd.read_csv(sample_files[0])
        print(f"Using sample data shape: {df.shape}")
        
        # Apply comprehensive filtering pipeline to sample data
        print("\nApplying comprehensive filtering pipeline to sample data...")
        print("Parameters optimized for head-inclusive motion at 2000 Hz:")
        print("- High-pass Butterworth: 0.3 Hz, order = 2")
        print("- Low-pass Butterworth: 30 Hz, order = 4")
        print("- Median filter: kernel = 11 samples (~5.5 ms)")
        print("- Savitzky-Golay: window = 51 samples (~25 ms), polyorder = 3")
        print("- All filters use filtfilt for zero-phase filtering")
        
        df_filtered = apply_filtering_pipeline(
            df, 
            sampling_rate=2000.0,
            columns=['x', 'y', 'z']
        )
        
        print(f"Final filtered shape: {df_filtered.shape}")
        
        # Save filtered sample data
        filtered_dir.mkdir(parents=True, exist_ok=True)
        output_file = filtered_dir / f"filtered_sample_{sample_files[0].stem}.csv"
        df_filtered.to_csv(output_file, index=False)
        print(f"Saved filtered sample data to: {output_file}")
        
        # Show sample of filtered data
        print("\nSample filtered data:")
        print(df_filtered.head())
    else:
        print("No sample data found either.")


Found 1 daily aggregated files
Daily data shape: (1, 9)
Columns: ['athlete_id', 'date', 'r_gs_mean', 'r_gs_max', 'r_gs_std', 'sample_count', 'x_mean', 'y_mean', 'z_mean']

Applying comprehensive filtering pipeline...
Parameters optimized for head-inclusive motion at 2000 Hz:
- High-pass Butterworth: 0.3 Hz, order = 2
- Low-pass Butterworth: 30 Hz, order = 4
- Median filter: kernel = 11 samples (~5.5 ms)
- Savitzky-Golay: window = 51 samples (~25 ms), polyorder = 3
- All filters use filtfilt for zero-phase filtering
Step 1: Applying high-pass filter (0.3 Hz, order=2)...
Step 2: Applying low-pass filter (30 Hz, order=4)...
Step 3: Applying median filter (kernel=11)...
Step 4: Applying Savitzky-Golay filter (window=51, polyorder=3)...
Filtering pipeline complete!
Final filtered shape: (1, 21)
Saved filtered data to: ../data/05_filtering/filtered_data/filtered_daily_agg_sample_imu_A002_R001.csv

Sample filtered data:
  athlete_id        date  r_gs_mean  r_gs_max  r_gs_std  sample_count  \


# 05: Filtering (HP, LP, Rolling Average)

Apply high-pass, low-pass, and rolling average filters to IMU data.
